# Add a new plugin

By default, the library will import all the files with prefix "plugin\_\*.py" from src/synthcity/plugins, and load all the classes which implement the [Plugin interface](src/synthcity/plugins/core/plugin.py).

Each plugin must implement the following methods:
- hyperparameter_space() - a static method that returns the hyperparameters that can be tuned during AutoML.
- type() - a static method that returns the type of the plugin. e.g., debug, generative, bayesian, etc.
- name() - a static method that returns the name of the plugin. e.g., ctgan, random_noisee, etc.
- _fit() - internal method, called by `fit` on each training set.
- _generate() - internal method, called by `generate`.

## Existing plugins

In [1]:
from synthcity.plugins import Plugins

generators = Plugins()

generators.list()

['pategan',
 'copulagan',
 'privbayes',
 'bayesian_network',
 'tvae',
 'nflow',
 'ctgan',
 'gaussian_copula',
 'adsgan',
 'rtvae']

## Example plugin: Generate 0-1

In [2]:
# stdlib
from typing import Any, List

# third party
import pandas as pd
import numpy as np

# synthcity absolute
from synthcity.plugins.core.distribution import Distribution
from synthcity.plugins.core.plugin import Plugin
from synthcity.plugins.core.schema import Schema


class ZeroOnePlugin(Plugin):
    """Dummy plugin for debugging.
    """

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @staticmethod
    def name() -> str:
        return "zero_one"

    @staticmethod
    def type() -> str:
        return "debug"

    @staticmethod
    def hyperparameter_space(*args: Any, **kwargs: Any) -> List[Distribution]:
        return []

    def _fit(self, X: pd.DataFrame, *args: Any, **kwargs: Any) -> "ZeroOnePlugin":
        self.features_count = X.shape[1]
        return self

    def _generate(self, count: int, syn_schema: Schema, **kwargs: Any):
        return np.random.randint(0, 2, size=(count, self.features_count))

In [3]:
# Add the new plugin to the collection

generators.add("zero_one", ZeroOnePlugin)

In [4]:
# Check the new plugins list
generators.list()

['pategan',
 'copulagan',
 'privbayes',
 'bayesian_network',
 'tvae',
 'nflow',
 'ctgan',
 'gaussian_copula',
 'adsgan',
 'rtvae']

In [5]:
# Load reference data

from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True, as_frame=True)

X

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [6]:
# Train the new plugin

gen = generators.get("zero_one")

gen.fit(X)

In [7]:
# Generate some new data

gen.generate(count=10)

RuntimeError: Plugin zero_one failed to meet the synthetic constraints.

### Oops, this didn't work.

__The Plugin interface enforces the new generated data to:__
 - satistify the same constraints as the training set.
 - Or to satisfy the constraints provided at inference time(if provided).
 
 
 If the generated dataframe fails to comply, an exception will be raised.

Let's try again

## A functional plugin

In [8]:
# stdlib
from typing import Any, List

# third party
import pandas as pd
import numpy as np

# synthcity absolute
from synthcity.plugins.core.distribution import Distribution
from synthcity.plugins.core.plugin import Plugin
from synthcity.plugins.core.schema import Schema


class DummyGeneratorPlugin(Plugin):
    """Dummy plugin for debugging.
    """

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @staticmethod
    def name() -> str:
        return "dummy_generator"

    @staticmethod
    def type() -> str:
        return "debug"

    @staticmethod
    def hyperparameter_space(*args: Any, **kwargs: Any) -> List[Distribution]:
        return []

    def _fit(self, X: pd.DataFrame, *args: Any, **kwargs: Any) -> "ZeroOnePlugin":
        return self

    def _generate(self, count: int, syn_schema: Schema, **kwargs: Any):
        result = self.schema().sample(count)
        result[syn_schema.features()] = syn_schema.sample(count)

        return result

In [9]:
generators.add("dummy_generator", DummyGeneratorPlugin)

generators.list()

['pategan',
 'copulagan',
 'privbayes',
 'bayesian_network',
 'tvae',
 'nflow',
 'ctgan',
 'gaussian_copula',
 'adsgan',
 'rtvae']

In [10]:
# Train the new plugin

gen = generators.get("dummy_generator")

gen.fit(X)

In [11]:
# Generate some new data

gen.generate(count=10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,13.664722,21.034517,43.931499,709.228309,0.147721,0.175627,0.168795,0.142350,0.170785,0.055169,...,27.536344,39.657085,207.783026,912.391323,0.210903,0.747402,0.840612,0.013928,0.609326,0.174273
1,14.421224,20.056507,154.983758,476.934941,0.133020,0.076152,0.119755,0.004475,0.127397,0.080556,...,12.956999,21.750595,99.500232,2913.859400,0.153913,0.636995,0.171048,0.162316,0.169887,0.193868
2,24.658198,33.915963,101.475827,1923.145847,0.114865,0.232108,0.349729,0.034488,0.280795,0.082932,...,28.158515,40.165518,224.039002,3055.525586,0.117808,0.330002,1.221945,0.075885,0.323335,0.114793
3,24.785912,31.818250,62.554238,2186.661486,0.118217,0.176044,0.404303,0.156156,0.118351,0.072483,...,27.922236,43.554788,88.016923,3095.329085,0.215137,0.914480,0.927985,0.025265,0.643123,0.111601
4,15.218210,21.543154,160.719452,381.275876,0.095165,0.223477,0.407358,0.036057,0.289392,0.066121,...,21.999230,32.879602,60.009386,3605.819908,0.102308,0.598882,1.207180,0.024298,0.230351,0.144713
5,12.141894,12.509685,82.627654,1662.219797,0.135404,0.206103,0.280147,0.181144,0.283651,0.051640,...,24.944995,43.943448,232.358482,3225.661409,0.123367,0.737670,1.144321,0.143129,0.262574,0.094060
6,13.703455,9.847313,108.575764,1379.477663,0.113807,0.102132,0.213451,0.087087,0.129837,0.050775,...,17.198100,45.307433,167.668213,4183.117152,0.101735,0.958365,0.075931,0.286271,0.420871,0.117224
7,20.795177,34.393961,85.478112,2386.601428,0.152420,0.265247,0.322765,0.079728,0.139432,0.067352,...,8.958008,21.396377,81.327397,3054.308601,0.092307,0.195326,0.923691,0.192111,0.259356,0.067861
8,12.242826,24.909209,158.195013,2221.357106,0.095565,0.321461,0.160267,0.181677,0.280496,0.079830,...,19.201609,35.739956,242.188526,1491.167363,0.104695,0.426686,0.078203,0.015531,0.260088,0.132303
9,21.013094,25.030682,98.619060,2090.446402,0.140983,0.125449,0.071439,0.135802,0.296431,0.055725,...,15.701416,29.633967,128.731449,1909.093053,0.099830,0.662588,0.297903,0.078132,0.444995,0.088048


In [12]:
# Custom generation constraints

from synthcity.plugins.core.constraints import Constraints

constraints = Constraints(rules=[("worst radius", ">", 15)])

generated = gen.generate(count=10, constraints=constraints)

assert (generated["worst radius"] > 15).any()

generated

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,27.885375,38.207181,110.603136,1663.480867,0.134178,0.192541,0.142879,0.139134,0.284518,0.052314,...,22.003593,25.143597,198.669083,3618.119676,0.095340,0.535659,0.663335,0.106536,0.637307,0.070497
1,25.754514,27.960296,132.516041,579.827658,0.124789,0.206966,0.103095,0.182913,0.208961,0.084479,...,23.879545,42.866674,235.276455,1751.668495,0.159409,0.223755,1.243697,0.246881,0.589260,0.068706
2,21.349283,16.225610,115.144775,563.685133,0.115223,0.064825,0.204125,0.185057,0.126735,0.062296,...,29.819044,46.012185,237.709270,375.494950,0.134313,0.260939,0.011538,0.121729,0.201399,0.207170
3,26.956580,16.260340,61.654215,475.659936,0.103964,0.283667,0.159340,0.054388,0.131589,0.067503,...,22.578051,33.892807,243.741160,4094.309702,0.103341,0.926496,1.235334,0.256227,0.193976,0.140475
4,24.900086,9.856374,119.993574,2374.927535,0.109853,0.067930,0.320892,0.028996,0.166049,0.052673,...,28.535667,47.882388,60.841194,1898.372905,0.183672,0.982050,0.629901,0.164130,0.300707,0.197303
5,15.756032,30.008491,99.063584,1366.036588,0.146727,0.269427,0.102425,0.194255,0.178996,0.068912,...,22.250979,36.564820,85.419043,3462.790337,0.120517,0.132785,0.032995,0.189223,0.181184,0.147377
6,19.528836,20.980511,120.234708,200.475583,0.071224,0.055272,0.345958,0.043508,0.236842,0.068620,...,30.177014,18.386058,110.904316,1415.760650,0.108686,0.632983,0.513693,0.070858,0.407449,0.157154
7,27.575997,34.133246,188.364951,369.039544,0.131195,0.238926,0.388497,0.007383,0.248863,0.056800,...,27.580837,12.600590,241.683651,245.768474,0.187578,0.468288,0.829186,0.280408,0.242938,0.189937
8,20.713949,28.049310,145.350443,1738.616309,0.106103,0.290624,0.426281,0.097133,0.279931,0.057718,...,24.837739,46.178606,232.844366,624.003913,0.219369,0.276880,0.895063,0.101784,0.216594,0.058510
9,15.040789,21.051513,75.331360,2317.227729,0.156715,0.271790,0.052687,0.020880,0.159065,0.054877,...,35.058260,21.287615,219.253845,2881.825753,0.193798,0.154889,0.031730,0.101481,0.167592,0.084040
